### ▶齊藤先生の正解データを座標入りで保存（⇒../data/shape_grandtruth_data.pkl）

##### ▶ライブラリ

In [1]:
import pandas as pd
import numpy as np
import requests
import io
import math
import copy
from scipy.stats import gmean
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import patches
from pipeline_functions import PipelineFunctions
pf = PipelineFunctions()
from view_functions import ViewFunctions
vf = ViewFunctions()

import umap
from sklearn.manifold import TSNE
import seaborn as sns
palette = ['#CC521D', '#4F4AD7', '#39AE3D']

##### ▶データを読み込む

In [2]:
# 解析対象データ
target_info = pd.read_excel('../data/all_drawing_info.xlsx', index_col=0).query('is_asai == 1')
box_info = pd.read_excel('../data/all_drawing_info.xlsx', index_col=0).query('is_preanalysis_box == 1')
bag_info = pd.read_excel('../data/all_drawing_info.xlsx', index_col=0).query('is_preanalysis_bag == 1')
info = pd.concat([box_info, bag_info], axis=0)

# 正解データ
ground_truth = pd.read_excel('../data/shape_groundtruth_id.xlsx', index_col=0)
gt_straight = ground_truth.query('shape == "straight"')
gt_curve = ground_truth.query('shape == "curve"')
gt_complex = ground_truth.query('shape == "complex"')

In [3]:
drawing_ids = []
rotate_types = []
stroke_ids = []
shapes = []
rotated_xs = []
rotated_ys = []
normalized_xs = []
normalized_ys = []

for did in target_info['drawing_id']:
    # 重複なし，回転済み，サイズ正規化済みのdfを取得
    rotate_type = target_info.query('drawing_id == @did')['rotate_type'].iloc[0]
    df = pf.get_normalized_data(f'../data/all_tsv/id_{did}.tsv', rotate_type)
    
    for sid, shape, rx, ry, nx, ny in zip(
    df['stroke_id'], df['shape_int'], 
    df['rotated_x'], df['rotated_y'], 
    df['size_unified_x'], df['size_unified_y']
    ):
        drawing_ids.append(did)
        rotate_types.append(rotate_type)
        stroke_ids.append(sid)
        shapes.append(shape)
        rotated_xs.append(rx)
        rotated_ys.append(ry)
        normalized_xs.append(nx)
        normalized_ys.append(ny)
        
coordsizes = [len(coord) for coord in rotated_xs]

In [4]:
all_stroke = pd.DataFrame({
    'drawing_id': drawing_ids,
    'stroke_id': stroke_ids,
    'shape_int': shapes,
    'rotate_type': rotate_types,
    'pt_cnt': coordsizes,
    'rotated_x': rotated_xs,
    'rotated_y':rotated_ys,
    'normalized_x': normalized_xs,
    'normalized_y': normalized_ys
})

In [34]:
straight = pd.concat([all_stroke.query('drawing_id == @did').query('stroke_id == @sid') for did, sid in zip(gt_straight['drawing_id'], gt_straight['stroke_id'])], axis=0)
straight['saito_label'] = ['straight' for i in range(len(straight))]
curve = pd.concat([all_stroke.query('drawing_id == @did').query('stroke_id == @sid') for did, sid in zip(gt_curve['drawing_id'], gt_curve['stroke_id'])], axis=0)
curve['saito_label'] = ['curve' for i in range(len(curve))]
complex = pd.concat([all_stroke.query('drawing_id == @did').query('stroke_id == @sid') for did, sid in zip(gt_complex['drawing_id'], gt_complex['stroke_id'])], axis=0)
complex['saito_label'] = ['complex' for i in range(len(complex))]

data = pd.concat([straight, curve, complex], axis=0)

##### ▶2値のラベルを追加

In [35]:
# 齊藤先生のラベル
is_good_saito = []
for label in data['saito_label']:
    if label == 'straight' or label == 'curve':
        is_good_saito.append(1)
    else:
        is_good_saito.append(0)
data['is_good_saito'] = is_good_saito

# 既存の線種判定のラベル
is_good_rulebase = []
for label in data['shape_int']:
    if label <= 2:
        is_good_rulebase.append(1)
    else:
        is_good_rulebase.append(0)
data['is_good_rulebase'] = is_good_rulebase

In [42]:
data.to_pickle('../data/shape_groudtruth_data.pkl')